# Hyperparameters training for Gaussian Process kernels

This code compares the kernel's hyperparameters  trained using 3 codes:
* SKlearn
* GPy
* TFGP - **tfgp_trainer** was written for SLAC National Lab, and uses Tensorflow.   

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import time
import warnings
import numpy as np
warnings.filterwarnings("ignore")

# Load data

For this example we load exprimental data from SPEAR3 storage ring at SLAC National Lab.
https://www-ssrl.slac.stanford.edu/spear3/

* X: Input include 13 controls (quadropole current in [mA]).
* Y: Objective includes the beam loss rate [mA/min].


In [10]:
df = pd.read_excel('Data/spear_data.xlsx')
X=df.iloc[:,0:13] 
Y=df['objective']
print('Y.shape',Y.shape,'\nX.shape',X.shape)

Y.shape (120,) 
X.shape (120, 13)


In [32]:
temp = np.load()

,0,1,2,3,4,5,6
0,1498.0,0.0,390000.0,-654100.0,-3.751500e+06,119931.5,648691.415
1,1498.0,0.0,390000.0,-654100.0,-3.751480e+06,119931.5,648691.415
2,1498.0,0.0,390000.0,-654100.0,-3.751460e+06,119931.5,648691.415
3,1498.0,0.0,390000.0,-654100.0,-3.751439e+06,119931.5,648691.415
4,1498.0,0.0,390000.0,-654100.0,-3.751419e+06,119931.5,648691.415
...,...,...,...,...,...,...,...
95,1498.0,0.0,390000.0,-654100.0,-3.749581e+06,119931.5,648691.415
96,1498.0,0.0,390000.0,-654100.0,-3.749561e+06,119931.5,648691.415
97,1498.0,0.0,390000.0,-654100.0,-3.749540e+06,119931.5,648691.415
98,1498.0,0.0,390000.0,-654100.0,-3.749520e+06,119931.5,648691.415


In [16]:
# combine the data from linescans and the data from a test GP run

x = np.load('Linescans/HighCs_3fold_emit_WhiteNoise/OBJ_x.npy')
x = np.concatenate((x, np.load('Obj_x_GPrun.npy')),axis=0)
print(x.shape)
X = pd.DataFrame(x)

y = np.load('Linescans/HighCs_3fold_emit_WhiteNoise/OBJ_y.npy')
y = np.concatenate((y, np.load('Obj_y_GPrun.npy')),axis=0)
print(y.shape)
Y = pd.DataFrame(y).squeeze()

print('Y.shape',Y.shape,'\nX.shape',X.shape)

(901, 7)
(901,)
Y.shape (901,) 
X.shape (901, 7)


# Training using TFGP trainer

In [17]:
from Modules.tfgp_trainer import tfgp_trainer
tf.compat.v1.disable_eager_execution()

In [18]:
Niter = 1200

t0 = time.time()
tfgp = tfgp_trainer(X, Y,verbose=0)

tfgp.train(lr=0.005, 
           niter=Niter,
           optimizer=[tf.compat.v1.train.AdamOptimizer, tf.optimizers.SGD, tf.optimizers.RMSprop][0],
           gradient_clipping=0,
           kernel=['rbf','matern32','matern52'][0],
           correlations = False,
           offset = False,
           monitor_period=None,
           monitor_gradient=None) 

tfgp_t = time.time()-t0
print ('took ',tfgp_t ,' seconds')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


TypeError: __init__() got an unexpected keyword argument 'max_value'

We can then plot results to see behaviour during the optimization and convergance.

In [6]:
from Modules.utils import GP_plot_results
tfgp_results = GP_plot_results(tfgp,Y)

IndexError: list index out of range

#  Training using SKlearn

In [38]:
from Modules.sk_trainer import sk_trainer
sk=sk_trainer()
sk.sk_train(X, Y, kernels=['rbf'], offset=False, verboseQ=True, saveQ=False)

********* 
 rbf
took  30.863511562347412  seconds
Inital kernel: 3.16**2 * RBF(length_scale=[1, 1, 1, 1, 1, 1, 1]) + WhiteKernel(noise_level=0.01)
Learned kernel: 0.285**2 * RBF(length_scale=[1.87e-07, 0.000757, 20, 1.97e-06, 7.24e-05, 0.00223, 0.0313]) + WhiteKernel(noise_level=1e-08)
Log-marginal-likelihood: -147.532


#  Training using to GPy

In [39]:
from Modules.gpy_trainer import gpy_trainer
gpy=gpy_trainer()
gpy.gpy_train(X, Y[:,None], kernels= ['rbf'], max_iters=5000, offset=False, verboseQ=True, saveQ=False)

********* 
 rbf
Optimization restart 1/5, f = 149.6705733152221
Optimization restart 2/5, f = 149.67057331522187
Optimization restart 3/5, f = 149.670573315222
Optimization restart 4/5, f = 149.67057331522238
Optimization restart 5/5, f = 149.67057331522204
took  5.981610536575317  seconds
Log-marginal-likelihood: -149.671


# Comparison of hyperparameters from all codes

In [20]:
# collect the results into a pandas dataframe to display
pd.DataFrame({"Param": ['time','negative log likelihood', 'amplitude', 'noise', 'offset'],
              "tfgp": [tfgp_t, tfgp_results['ll'], tfgp_results['amp_param'], tfgp_results['noise_param_variance'] , tfgp_results['offset_param']],
              "SK": [sk.sk_t, sk.results['ll'], sk.results['amp_param'], sk.results['noise_param_variance'] , sk.results['offset_param']],
              "GPy": [gpy.gpy_t, gpy.results['ll'], gpy.results['amp_param'], gpy.results['noise_param_variance'] , gpy.results['offset_param']],
             })


NameError: name 'tfgp_t' is not defined

In [21]:
print('Lengthscales Comparison')
pd.DataFrame({"": X.keys(),
              "tfgp": tfgp_results['length_scale_param'],
              "SK": sk.results['length_scale_param'] ,
              "GPy": gpy.results['length_scale_param']
             })

Lengthscales Comparison


NameError: name 'tfgp_results' is not defined

In [40]:
# collect the results into a pandas dataframe to display
pd.DataFrame({"Param": ['time','negative log likelihood', 'amplitude', 'noise', 'offset'],
              "SK": [sk.sk_t, sk.results['ll'], sk.results['amp_param'], sk.results['noise_param_variance'] , sk.results['offset_param']],
              "GPy": [gpy.gpy_t, gpy.results['ll'], gpy.results['amp_param'], gpy.results['noise_param_variance'] , gpy.results['offset_param']],
             })

,Param,SK,GPy
0,time,3.086351e+01,5.981611
1,negative log likelihood,-1.475324e+02,-149.670573
2,amplitude,8.139119e-02,0.068528
3,noise,1.000000e-08,0.005204
4,offset,NaN,NaN


In [41]:
print('Lengthscales Comparison')
pd.DataFrame({"": X.keys(),
              "SK": sk.results['length_scale_param'] ,
              "GPy": gpy.results['length_scale_param']
             })

Lengthscales Comparison


,,SK,GPy
0,0,1.869406e-07,0.127393
1,1,7.573027e-04,0.689572
2,2,2.000000e+01,1.554374
3,3,1.969035e-06,0.490309
4,4,7.237757e-05,0.872161
5,5,2.232187e-03,0.653022
6,6,3.134143e-02,1.610226
